In [1]:
import re
import time
import json
import numpy as np
from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

 
# service = Service('chromedriver')
# service.start()
# options = Options() #webdriver.chromeOptions()

# prefs={"download.default_directory":"./EnterpriseFiles",
#       "download.prompt_for_download": False,
#       "download.directory_upgrade": True,
#       "safebrowsing.enabled": True  }

# options.add_experimental_option("prefs",prefs)

# driver = webdriver.Firefox(executable_path='geckodriver.exe')*

In [2]:
# create webdriver object
driver = webdriver.Chrome(executable_path="../chromedriver.exe")

In [340]:
# load the page
driver.get("https://data.inpi.fr")

    #make the request

req = driver.find_element_by_class_name("search-input")
req.clear()
req.send_keys("Laurent Mignon")
req.submit()

time.sleep(5)

#     driver.maximize_window()

check_all_result = driver.find_element_by_xpath("//input[@type='checkbox'][@id='result-all']")#[contains(@class='checkbox-result-bloc')]
driver.execute_script("arguments[0].click();", check_all_result)
print("ok 1")

download_button = driver.find_element_by_xpath("//button[@title='Télécharger'][@class='btn btn-link p-0 actions-buttons']")
driver.execute_script("arguments[0].click();", download_button)
print("ok 2")

csv_selector = driver.find_element_by_id("export_companies_format_1")
driver.execute_script("arguments[0].click();", csv_selector)
print("ok 3")

# select_all_infos = driver.find_element_by_xpath("//input[@type='checkbox'][@name='select-all'][@class='select-all-input']")
select_all_infos = driver.find_element_by_xpath("//label[@for='select-all'][@class='select-all-label ml-1']")
driver.execute_script("arguments[0].click();", select_all_infos)
print("ok 4")

download = driver.find_element_by_id('export_companies_submit')
time.sleep(1)
download.click()
# driver.execute_script("arguments[0].clcik();", download)
print('downloading')

ok 4
downloading


In [153]:
#     get the link on the page
# elems = driver.find_elements(By.XPATH, "//a[contains(@class,'not-link')]")
# links = list(set([elem.get_attribute('href') for elem in elems]))
# print(f'Scraper got {len(links)} links')

# all_keys = driver.find_elements_by_xpath("//p[contains(@class,'mb-0 font-weight-300 font-size-0-9-rem')]")
# all_values = driver.find_elements(By.XPATH,"//p[contains(@class, 'font-size-0-9-rem')]")

# values = get_attr(all_values)

# with open('values.txt','w') as f:
#     for _ in all_values:
#         f.write(_.get_attribute("innerHTML"))

In [20]:
def getNbPages(driver):
    all_pages = driver.find_elements(By.XPATH,"//a[@role='button'][contains(@class,'mr-3 pointer page-number ')]")
    return len(all_pages)

def getLinksFromPage(driver):
    elems = driver.find_elements(By.XPATH, "//a[contains(@class,'not-link')]")
    links = list(set([elem.get_attribute('href') for elem in elems]))
    return links

def getPagesLinks(nom,prenom,driver,nb_pages=3, all_pages=False):

    all_links = []
    l1 = 'https://data.inpi.fr/search?advancedSearch=%257B%257D&filter=%257B%257D&nbResultsPerPage=250&order=asc'
    l2 = "&page=1&q="+prenom+'%20'+nom+'&sort=relevance&type=companies'
    link = l1+l2
    driver.get(link)
    time.sleep(7)
    nb_ = getNbPages(driver)

    all_links.append(getLinksFromPage(driver))
    
    if all_pages or (nb_pages>= nb_):
        for i in range(2,nb_+1):
            driver.get(link.replace('page=1',f'page={i}'))
            time.sleep(7)
            all_links.append(getLinksFromPage(driver))
    else:
        for i in range(2,nb_pages+1):
            driver.get(link.replace('page=1',f'page={i}'))
            time.sleep(7)
            all_links.append(getLinksFromPage(driver))
    
    all_links = [lk for page_likns in all_links for lk in page_likns]

    return all_links


def get_attr(web_elements):
    # return [_.get_attribute("innerHTML") for _ in web_elements ]
    return [el.text for el in web_elements] 

def list_dict_from_page(driver):
    all_values = driver.find_elements(By.XPATH,"//p[contains(@class, 'font-size-0-9-rem')]")
    values = get_attr(all_values)
    
    identity=[]
    representants =[]
    etablissements=[]
    for i in range(len(values)):
        if 'Pour plus d\'informations sur les représentants' in values[i]:
            identity = values[0:i]

        if "Type d'établissement" in values[i]:
            representants = values[len(identity)+1:i]   
            etablissements=values[i:]
            break

    return identity,representants,etablissements

def get_dict_identity(identity):
    header = {}
    for i in range(0,len(identity),2):
        header.update({identity[i]:identity[i+1]})
        
    return header

def getIndicesToDelete(keys):
    return [i for i, j in enumerate(keys) if j == "Nom d'usage"]

def dropNomDusage(ls, indices_to_delete):
    clean_list = np.delete(ls, indices_to_delete).tolist()
    return clean_list

def getTheDict(keys,n_):
    TheDict = {}

    for j in range(0,len(keys),n_):
        try:
            i=next(ls)
            rep[i] = {k:v for (k,v) in zip(keys[j:j+n_],values[j:j+n_])}
        except:
            break
        
    return TheDict


def get_dict_representative(representatives):

    rep = {}
    representatives = [re.sub('<\w*>|\n|</em>|<em class="highlight-elasticsearch">', '',_).strip() for _ in representatives] 
    nb_rep_ = representatives.count('Nom, Prénom(s)')
    
    if nb_rep_==0:
        return {"Zero Message":'Jiraya founds no representatives'}

    else:
        
        ls = (k for k in range(1,nb_rep_+1))

        keys = [representatives[i] for i in range(0,len(representatives),2)]
        indices_to_delete = getIndicesToDelete(keys)
        keys = dropNomDusage(keys,indices_to_delete)
        values = dropNomDusage([representatives[i] for i in range(1,len(representatives),2)],indices_to_delete)

        nb_rep_attr_ = int(len(keys)/nb_rep_)
        
        for j in range(0,len(keys),nb_rep_attr_):
            try:
                i=next(ls)
                rep[i] = {k:v for (k,v) in zip(keys[j:j+nb_rep_attr_],values[j:j+nb_rep_attr_])}
            except:
                break

        # rep = getTheDict(keys,nb_rep_attr_)
        return  rep

def get_dict_benef(driver):
    all_keys = driver.find_elements(By.XPATH,"//p[contains(@class, 'mb-0 font-size-13 inpi-light')]")
    all_values = driver.find_elements(By.XPATH,"//p[contains(@class, 'font-size-15')]")

    keys = get_attr(all_keys)
    values = get_attr(all_values)

    nb_benef_ = keys.count('Nom prénom')
    if nb_benef_==0:
        return {'Zero Message':'Jiraya founds no beneficiaries'}
    else : 
        nb_benf_attr_ = int(len(keys)/nb_benef_)

        beneficiaires = {}
        ls = (k for k in range(1,nb_benef_+1))
    
        for j in range(0,len(keys),nb_benf_attr_):
            try:
                i=next(ls)
                beneficiaires[i] = {k:v for (k,v) in zip(keys[j:j+nb_benf_attr_],values[j:j+nb_benf_attr_])}
            except:
                break

        return beneficiaires

def get_dict_ets(etablissements):

    keys = [etablissements[i] for i in range(0,len(etablissements),2)]
    values = [etablissements[i] for i in range(1,len(etablissements),2)]

    nb_ets_ = keys.count("Type d'établissement")
    if nb_ets_ ==0:
        return {'Zero message': 'Jiraya founds no etablissement'}
    else:
            
        nb_ets_attr_ = int(len(keys)/nb_ets_)

        etablissements = {}
        ls = (k for k in range(1,nb_ets_+1))
    
        for j in range(0,len(keys),nb_ets_attr_):
            try:
                i=next(ls)
                etablissements[i] = {k:v for (k,v) in zip(keys[j:j+nb_ets_attr_],values[j:j+nb_ets_attr_])}
            except:
                break

        return etablissements

def dict_from_page(identity,representatives,driver,etablissements):
    TheDict = { "identity": get_dict_identity(identity),
                "representatives" : get_dict_representative(representatives),
                "beneficiaires": get_dict_benef(driver),
                "etablissements": get_dict_ets(etablissements)
                                    }
    return TheDict


def run(nom,prenom,driver,all_pages=False):
    links = getPagesLinks(nom,prenom,driver,nb_pages=1, all_pages=all_pages)
    all_enterprise = dict()
    for i in range(len(links)):
        driver.get(links[i])
        identity,representatives,etablissements = list_dict_from_page(driver)
        all_enterprise[i] = dict_from_page(identity,representatives,driver,etablissements)

    return all_enterprise



In [ ]:
run('Mignon',"Laurent",driver,all_pages=False)

In [320]:
trial = json.dump(test, open("test.json","w"))

In [7]:
from main_scraper import inpi_data
result = inpi_data("Mignon","Laurent").run()

In [21]:
lk = getPagesLinks('DIDIER','SEBASTIEN',driver,nb_pages=3, all_pages=False)

In [22]:
len(lk)

500